# Using LoadFilteredEntitySetData endpoint

This tutorial walks through how to use the load filtered entity set data endpoint. If you're using the API directly, you will use:

`POST <baseurl>/datastore/data/set/<entitySetId>/filtered`

where the base url is either api.openlattice.com or integration.openlattice.com.

## Possible request bodies:

For a default search, (paging back max of 10000 results):
`{}` 

To specify the total number of hits:
```
{
    "pageSize" : 100
}
```

To get the next page of results, you would take the entityKeyId of the last entity returned on the last page:
```
{
  "bookmarkId": <last entityKeyId>
}
```
If you want to use the filter, specify the propertyTypeId that you want to filter on, as well as the filter itself. For example, this this is using the DateTimeRangeFilter:

```
{
    "propertyTypeId": "0ee3acba-51a7-4f8d-921f-e23d75b07f65",
    "filter": {
        "@class": "com.openlattice.analysis.requests.DateTimeRangeFilter",
        "lowerbound": "2000-10-29T01:19:12-04:00"
    }
```

This would filter for dates past 10/29/2000.

Or, if you want to use a string filter, you can use:

```
{
    "propertyTypeId": "5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2",
    "filter": {
        "@class": "com.openlattice.analysis.requests.StringValueFilter",
        "values": ["Woman"]
    }
}
```
This would filter for all values that include "Woman".

## Python walkthrough

To follow along, you will need to install [olpy](https://github.com/openlattice/olpy) and [openlattice](https://github.com/Lattice-Works/api-clients/tree/master/python). [This](https://help.openlattice.com/article/104-overview-and-installation-of-api-clients) is a detailed setup instructions for setting up the openlattice module.

Then set up the configurations.

In [1]:
import openlattice
import olpy
conf = openlattice.Configuration()
conf.access_token = olpy.get_jwt()
conf.host = "https://api.openlattice.com"
dataAPI = openlattice.DataApi(openlattice.ApiClient(conf))

## Loading data

The following is a toy dataset derived in the [deleting entity sets tutorial](www.github.com/lattice-works/api-clients/examples/pipelines/deleting_entity_sets.ipynb). If you want to load the first 10,000 results of the dataset, use:

In [11]:
entset = "0f898abd-edfe-4212-8642-74db49efa474"
dataAPI.load_filtered_entity_set_data(
     entset, 
     openlattice.FilteredDataPageDefinition()
 )

[{'nc.SubjectIdentification': ['Child1'],
  'openlattice.@id': ['24220000-0000-0000-8000-00000001f3aa']},
 {'nc.SubjectIdentification': ['Woman1'],
  'openlattice.@id': ['25fc0000-0000-0000-8000-00000001e3e2']},
 {'nc.SubjectIdentification': ['Woman2'],
  'openlattice.@id': ['260a0000-0000-0000-8000-00000001e2a1']},
 {'nc.SubjectIdentification': ['Child2'],
  'openlattice.@id': ['26140000-0000-0000-8000-00000001f19f']}]

This returns a list of dictionaries. `openlattice.@id` is generated as a unique entity key with OpenLattice. The only "real" data here is `nc.SubjectIdentification`. 

Now, if you only wanted to load the first two datapoints, you will need to use:

In [12]:
filtered_pg = openlattice.FilteredDataPageDefinition(page_size = 2)
dataAPI.load_filtered_entity_set_data(entset, filtered_pg)

[{'nc.SubjectIdentification': ['Child1'],
  'openlattice.@id': ['24220000-0000-0000-8000-00000001f3aa']},
 {'nc.SubjectIdentification': ['Woman1'],
  'openlattice.@id': ['25fc0000-0000-0000-8000-00000001e3e2']}]

If you want to load the next two datapoints, you will need to use the last entity key (i.e. `openlattice.@id`) from the search results. In this case, that's `25fc0000-0000-0000-8000-00000001e3e2`.

In [13]:
filtered_pg = openlattice.FilteredDataPageDefinition(
    page_size = 2, 
    bookmark_id = "25fc0000-0000-0000-8000-00000001e3e2"
)
dataAPI.load_filtered_entity_set_data(entset, filtered_pg)

[{'nc.SubjectIdentification': ['Woman2'],
  'openlattice.@id': ['260a0000-0000-0000-8000-00000001e2a1']},
 {'nc.SubjectIdentification': ['Child2'],
  'openlattice.@id': ['26140000-0000-0000-8000-00000001f19f']}]

And you should see the last two entries!

If you want to filter on just `Woman1` for example, you can use:

In [14]:
fl = openlattice.Filter(
    _class = "com.openlattice.analysis.requests.StringValueFilter", 
    values = ["Woman1"]
)
filtered_pg = openlattice.FilteredDataPageDefinition(
    property_type_id = "5260cfbd-bfa4-40c1-ade5-cd83cc9f99b2", 
    filter = fl
)
dataAPI.load_filtered_entity_set_data(entset, filtered_pg)

[{'nc.SubjectIdentification': ['Woman1'],
  'openlattice.@id': ['25fc0000-0000-0000-8000-00000001e3e2']}]

To get the desired property type id, go to our [EDM page](https://openlattice.com/edm/#/propertyTypes). Search for a desired property type and the top right hand will have its corresponding UUID.

Another potentially useful filter is `"com.openlattice.analysis.requests.DateTimeRangeFilter"` with a `lowerbound` and `upperbound` arguments to filter through dates.